In [6]:
import sqlite3
import re
import pandas as pd

In [2]:
# Paths
sql_dump_path = "dump2_only_images.sql"   # your MySQL dump file
sqlite_db_path = "guide-mts-data.sqlite3" # your real sqlite DB

In [3]:
# Connect to your SQLite DB
conn = sqlite3.connect(sqlite_db_path)
cursor = conn.cursor()

insert_count = 0
skip_count = 0

In [4]:
# Regex to extract values from INSERT lines
pattern = re.compile(
    r"INSERT INTO\s+`?tbl_StuImg`?.*?\((\d+),\s*(0x[0-9A-Fa-f]+|NULL),\s*(\d+)\)",
    re.IGNORECASE
)

with open(sql_dump_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line.upper().startswith("INSERT"):
            continue
        
        match = pattern.search(line)
        if not match:
            skip_count += 1
            continue

        stu_id = int(match.group(1))
        pic_hex = match.group(2)
        # file_size = int(match.group(3))  # not needed for transfer

        if pic_hex is None or pic_hex.upper() == "NULL":
            continue

        # Convert MySQL hex "0xFFD8..." -> bytes
        pic_data = bytes.fromhex(pic_hex[2:])

        # Update customers table
        cursor.execute("""
            UPDATE customers
            SET customer_image = ?
            WHERE id = ?
        """, (pic_data, stu_id))

        insert_count += 1

conn.commit()
print(f"✅ Transferred {insert_count} images (skipped {skip_count} lines).")

✅ Transferred 4742 images (skipped 0 lines).


In [7]:
# Verify
verify_df = pd.read_sql_query(
    "SELECT id, length(customer_image) as img_size FROM customers WHERE customer_image IS NOT NULL LIMIT 10;",
    conn
)
display(verify_df)

conn.close()

,id,img_size
0,1,417037
1,2,424906
2,4,339934
3,5,422061
4,6,343991
5,7,386291
6,8,390839
7,9,418052
8,10,413864
9,11,154308
